In [29]:
from tinydb import TinyDB, Query
import json
import os

# Manage DB.
class Logos:
    def __init__(self, db_name: str) -> None:
        self.db_instance = TinyDB(db_name)
        self.fields = ["reality", "ideas", "metaphysics"]
        self.tables = {field: self.db_instance.table(field) for field in self.fields}

    def get_all(self, key: str, fields: list = []) -> list[dict]:
        search = self.tables[key].all()
        if fields:
            search = [{k: v for k, v in i.items() if k in fields} for i in search]
        return search
        

class Ontology(Logos):
    def __init__(self, db_name: str, custom_defaults_path: str = None) -> None:
        super().__init__(db_name)
        self.custom_defaults_path = custom_defaults_path
        self.DEFAULT = {
            "ideas": {
                "name": None,
                "id": None,
                "cat": "object",
                "desc": "An object.",
                "initRequire": [],
                "constRequire": [],
                "initGains": [],
                "constGains": [],
                "replace": [],
                "restrict": [],
                "visible": True,
                "turnToComplete": 0
            },
             "metaphysics": {
                "dismantleGain": 0.5,
                "turn": 1
            },
            "reality": {
                "id": None,
                "type": None,
                "amount": 1,
                "active": True
            }
        }

    # Set default values for partial entry.
    def set_default_values(self, obj: dict, defaults: dict) -> dict:
        for key in defaults.keys():
            if key not in obj.keys():
                obj.update({key: defaults[key]})
        return obj

    # Overwrite default values.
    def overwrite_defaults(self) -> None:
        try:
            with open(custom_defaults_path, "r") as f:
                self.DEFAULT = json.loads(f.read())
            
            print("Default values has been overwriten.")
        except OSError as e:
            raise Exception(e)

    def is_coherent(self, content: dict) -> (bool, dict, list):
        err = []
        for field in content.keys():
            if field not in self.fields:
                err.append(f"{field} is not valid structural keyword.")

            if field == "metaphysics":
                content[field] = [self.set_default_values(content[field], self.DEFAULT[field])]

            for element in [c for c in content[field] if field != "metaphysics"]:
                if "id" not in element.keys():
                    err.append(f"{element} need to have id key.")

                element = self.set_default_values(element, self.DEFAULT[field])
                    

        return not bool(err), content, err

    # Compile data into database.
    def compile_data(self, scenario_dir: str) -> None:
        valid_files = [f for f in os.listdir(scenario_dir) if f.endswith(".json")]

        for file in valid_files:
            with open(f"{scenario_dir}/{file}", "r") as f:
                content = json.loads(f.read())
                
            is_coherent, enriched, err = self.is_coherent(content)
            
            if is_coherent:
                [self.tables[k].insert_multiple(enriched[k]) for k in enriched.keys()]
            else:
                print(err)
                quit()
                    

In [30]:
ontology = Ontology("db.json")
ontology.compile_data("scenario")